<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# VaR methods

In [1]:
import numpy as np, pandas as pd, scipy as sp
from math import *
from scipy import stats
from scipy.stats import norm
import statsmodels.api as sm
from scipy import stats,random
import pandas_datareader.data as web
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="darkgrid")
p = print

In [2]:
# tickers = ['AAPL','IBM','MSFT']
# df = web.DataReader(tickers,'yahoo','01/01/2007','10/18/2019')
# rets = df['Adj Close'].pct_change().dropna()

In [3]:
confidence_level_99 = 0.99
z_99 = norm.ppf(1 - confidence_level_99)
p(z_99)
p(f'At a confidence level of 99%, z = {round(z_99, 2)}.')

-2.3263478740408408
At a confidence level of 99%, z = -2.33.


In [4]:
ticker = 'IBM'
n_shares=1000             # input 2
confidence_level=0.99     # input 3

z = norm.ppf(1-confidence_level) 
df = web.DataReader(ticker,'yahoo','01/01/2007','10/18/2019')

rets = df['Adj Close'].pct_change().dropna()

position= n_shares * df['Close'][-1] 
std = rets.std()

VaR = position * z * std
p(f'Holding = {round(position,3)} of {ticker} stock, VaR = {round(VaR,4)} tomorrow')

Holding = 134089.996 of IBM stock, VaR = -4361.2461 tomorrow


In [5]:
def normality_VaR(ticker_df):
    df = ticker_df
    n_shares=1000             # input 2
    confidence_level=0.99     # input 3

    z = norm.ppf(1-confidence_level) 
#     df = web.DataReader(ticker,'yahoo','01/01/2007','10/18/2019')

    rets = df['Adj Close'].pct_change().dropna()

    position= n_shares * df['Close'][-1] 
    std = rets.std()

    VaR = position * z * std
    p(f'Holding = {round(position,3)}, VaR = {round(VaR,4)} tomorrow')
    return

In [6]:
aapl_df = web.DataReader('AAPL','yahoo','01/01/2007','10/18/2019')

In [7]:
normality_VaR(aapl_df)

Holding = 236410.004, VaR = -10865.6966 tomorrow


In [8]:
msft_df = web.DataReader('MSFT','yahoo','01/01/2007','10/18/2019')

In [9]:
normality_VaR(msft_df)

Holding = 137410.004, VaR = -5459.8297 tomorrow


In [10]:
ticker = 'IBM'
n_shares = 1000             # input 2
confidence_level = 0.99     # input 3
nDays = 10

z = norm.ppf(1-confidence_level) 
ibm_df = web.DataReader(ticker,'yahoo','01/01/2007','10/18/2019')

rets = ibm_df['Adj Close'].pct_change().dropna()

position= n_shares * ibm_df['Close'][-1] 
std = rets.std() * np.sqrt(nDays)

VaR = position * z * std
p(f'Holding = {round(position,3)}, VaR = {round(VaR,4)} over {nDays} days')

Holding = 134089.996, VaR = -13791.4711 over 10 days


In [11]:
def normality_nDays_VaR(ticker_df, nDays):
    df = ticker_df
#     ticker = 'AAPL'
    n_shares = 1000             # input 2
    confidence_level = 0.99     # input 3
    nDays = nDays

    z = norm.ppf(1-confidence_level) 
#     df = web.DataReader(ticker,'yahoo','01/01/2007','10/18/2019')

    rets = df['Adj Close'].pct_change().dropna()

    position= n_shares * df['Close'][-1] 
    std = rets.std() * np.sqrt(nDays)

    VaR = position * z * std
    p(f'Holding = {round(position,3)}, VaR = {round(VaR,4)} over {nDays} day')
    return

In [12]:
normality_nDays_VaR(ibm_df,10)

Holding = 134089.996, VaR = -13791.4711 over 10 day


In [13]:
aapl_df = web.DataReader('AAPL','yahoo','01/01/2007','10/18/2019')

In [14]:
normality_nDays_VaR(aapl_df,10)

Holding = 236410.004, VaR = -34360.3496 over 10 day


In [15]:
msft_df = web.DataReader('MSFT','yahoo','01/01/2007','10/18/2019')

In [16]:
normality_nDays_VaR(msft_df,10)

Holding = 137410.004, VaR = -17265.4976 over 10 day


In [17]:
np.random.seed(1234)
n=5000000   

ret = random.normal(0,1,n)
p(f'mean = {np.mean(ret)}')
p(f'standard deviation = {np.std(ret)}')
p(f'skewness = {stats.skew(ret)}')
p(f'kurtosis = {stats.kurtosis(ret)}')

mean = -0.0002837106573708666
standard deviation = 1.0008485763507922
skewness = 0.0019744405393315747
kurtosis = 0.0015123186295351587


In [18]:
ticker = 'AAPL'            # input 1
n_shares = 1000            # input 2
confidence_level = 0.99   # input 3


# Method I: based on the first two moments
z = abs(norm.ppf(1-confidence_level)) 
df = web.DataReader(ticker,'yahoo','01/01/2007','10/18/2019')
rets = df['Adj Close'].pct_change().dropna()
position= n_shares * df['Close'][-1] 
mean = np.mean(rets)
std = np.std(rets)
VaR1 = position * (mean - z * std)
p(f'Holding = {round(position,2)} VaR = {round(VaR1,2)} over 1 day')

# Modified VaR: based on 4 moments
s = stats.skew(rets)
k = stats.kurtosis(rets)
t = z+1/6.*(z**2-1)*s+1/24.*(z**3-3*z)*k-1/36.*(2*z**3-5*z)*s**2
mVaR = position * (mean - t * std)
p(f'Holding = {round(position,2)}, modified VaR = {round(mVaR,2)} over 1 day')
# ('Holding=', 24853.46, 'VaR1=', -876.84, 'for 1 day ')
# ('Holding=', 24853.46, 'modified VaR=', -1500.41, 'for 1 day ')

Holding = 236410.0 VaR = -10588.68 over 1 day
Holding = 236410.0, modified VaR = -16654.77 over 1 day


In [19]:
aapl_df = web.DataReader('AAPL','yahoo','01/01/2007','10/18/2019')
normality_nDays_VaR(aapl_df,1)

Holding = 236410.004, VaR = -10865.6966 over 1 day


In [20]:
def modified_VaR(ticker_df):
    df = ticker_df        # input 1
    n_shares = 1000            # input 2
    confidence_level = 0.99   # input 3

    # Method I: based on the first two moments
    z = abs(norm.ppf(1-confidence_level)) 
#     df = web.DataReader(ticker,'yahoo','01/01/2007','10/18/2019')
    rets = df['Adj Close'].pct_change().dropna()
    position= n_shares * df['Close'][-1] 
    mean = np.mean(rets)
    std = np.std(rets)
    VaR1 = position * (mean - z * std)
    p(f'Holding = {round(position,2)}, VaR = {round(VaR1,2)} over 1 day')

    # Modified VaR: based on 4 moments
    s = stats.skew(rets)
    k = stats.kurtosis(rets)
    t = z+1/6.*(z**2-1)*s+1/24.*(z**3-3*z)*k-1/36.*(2*z**3-5*z)*s**2
    mVaR = position * (mean - t * std)
    
    p(f'Holding = {round(position,2)}, modified VaR = {round(mVaR,2)} over 1 day')
    return

In [21]:
ibm_df = web.DataReader('IBM','yahoo','01/01/2007','10/18/2019')

In [22]:
modified_VaR(ibm_df)

Holding = 134090.0, VaR = -4319.48 over 1 day
Holding = 134090.0, modified VaR = -6969.34 over 1 day


In [23]:
msft_df = web.DataReader('MSFT','yahoo','01/01/2007','10/18/2019')

In [24]:
modified_VaR(msft_df)

Holding = 137410.0, VaR = -5361.47 over 1 day
Holding = 137410.0, modified VaR = -11557.92 over 1 day


In [25]:
# VaR based on sorted historical returns

ticker='AAPL'
# To make our programs easier to understand, the time period is just 1 day:
z = norm.ppf(confidence_level) 
df = web.DataReader(ticker,'yahoo','01/01/2007','10/18/2019')
ret = df['Adj Close'].pct_change().dropna()

position = n_shares * df['Close'][-1] 
std=np.std(ret)

n = len(ret)
left_tail = int((1 - confidence_level) * n)
sorted_ret = np.sort(ret)


VaR = position * sorted_ret[left_tail]
p(f'Holding = {round(position,2)}, historical VaR = {round(VaR,4)} tomorrow')

Holding = 236410.0, historical VaR = -13224.7265 tomorrow


In [26]:
def historical_VaR(ticker_df):
    df = ticker_df
#     ticker = 'AAPL'
    n_shares = 1000             # input 2
    confidence_level = 0.99     # input 3
#     nDays = nDays

#     z = norm.ppf(1-confidence_level) 
#     df = web.DataReader(ticker,'yahoo','01/01/2007','10/18/2019')

    rets = df['Adj Close'].pct_change().dropna()
    position= n_shares * df['Close'][-1] 
    std = rets.std() #* np.sqrt(nDays)

    n = len(ret)
    left_tail = int((1 - confidence_level) * n)
    sorted_ret = np.sort(ret)

    VaR = position * sorted_ret[left_tail]
    p(f'Holding = {round(position,2)}, historical VaR = {round(VaR,4)} tomorrow')
    return

In [27]:
historical_VaR(aapl_df)

Holding = 236410.0, historical VaR = -13224.7265 tomorrow


In [28]:
normality_nDays_VaR(aapl_df,10)

Holding = 236410.004, VaR = -34360.3496 over 10 day


In [29]:
modified_VaR(aapl_df)

Holding = 236410.0, VaR = -10588.68 over 1 day
Holding = 236410.0, modified VaR = -16654.77 over 1 day


In [30]:
historical_VaR(ibm_df)

Holding = 134090.0, historical VaR = -7500.9665 tomorrow


In [31]:
modified_VaR(aapl_df)

Holding = 236410.0, VaR = -10588.68 over 1 day
Holding = 236410.0, modified VaR = -16654.77 over 1 day


In [32]:
historical_VaR(msft_df)

Holding = 137410.0, historical VaR = -7686.687 tomorrow


In [33]:
# Monte Carlo simulation VaR
# Monte Carlo simulation VaR
df = web.DataReader('WMT','yahoo','01/01/2007','10/18/2019')
ret = df['Adj Close'].pct_change()
mean=np.mean(ret)
std=np.std(ret)

sp.random.seed(12345) 
n_simulation=5000
n_shares = 1000             # input 2
confidence_level = 0.99 

ret2 = sp.random.normal(mean,std,n_simulation) 
ret3 = np.sort(ret2) 
m = int(n_simulation * (1-confidence_level))
VaR = position * (ret3[m])
p(f'Holding = {position} Monte Carlo VaR = {round(VaR,4)} tomorrow')

Holding = 236410.00366210938 Monte Carlo VaR = -6573.0393 tomorrow


In [34]:
def monte_carlo_VaR(ticker_df):
    # Monte Carlo simulation VaR
    sp.random.seed(12345) 
    n_simulation = 5000
    n_shares = 1000             # input 2
    confidence_level = 0.99 
    
    df = ticker_df
#     df = web.DataReader('WMT','yahoo','01/01/2007','10/18/2019')
    ret = df['Adj Close'].pct_change()
    mean=np.mean(ret)
    std=np.std(ret)
    position = n_shares * df['Close'][-1] 

    
    ret2 = sp.random.normal(mean,std,n_simulation) 
    ret3 = np.sort(ret2) 
    m = int(n_simulation * (1-confidence_level))
    VaR = position * (ret3[m])
    p(f'Holding = {round(position,2)} Monte Carlo VaR = {round(VaR,4)} tomorrow')
    return

In [35]:
monte_carlo_VaR(aapl_df)

Holding = 236410.0 Monte Carlo VaR = -10354.9889 tomorrow


In [36]:
monte_carlo_VaR(ibm_df)

Holding = 134090.0 Monte Carlo VaR = -4225.6823 tomorrow


In [37]:
monte_carlo_VaR(msft_df)

Holding = 137410.0 Monte Carlo VaR = -5244.0437 tomorrow


In summary:

In [38]:
normality_VaR(aapl_df)

Holding = 236410.004, VaR = -10865.6966 tomorrow


In [39]:
modified_VaR(aapl_df)

Holding = 236410.0, VaR = -10588.68 over 1 day
Holding = 236410.0, modified VaR = -16654.77 over 1 day


In [40]:
historical_VaR(aapl_df)

Holding = 236410.0, historical VaR = -7755.5248 tomorrow


In [41]:
monte_carlo_VaR(aapl_df)

Holding = 236410.0 Monte Carlo VaR = -10354.9889 tomorrow


In [42]:
normality_VaR(ibm_df)

Holding = 134089.996, VaR = -4361.2461 tomorrow


In [43]:
modified_VaR(ibm_df)

Holding = 134090.0, VaR = -4319.48 over 1 day
Holding = 134090.0, modified VaR = -6969.34 over 1 day


In [44]:
historical_VaR(ibm_df)

Holding = 134090.0, historical VaR = -4398.876 tomorrow


In [45]:
monte_carlo_VaR(ibm_df)

Holding = 134090.0 Monte Carlo VaR = -4225.6823 tomorrow


In [46]:
normality_VaR(msft_df)

Holding = 137410.004, VaR = -5459.8297 tomorrow


In [47]:
modified_VaR(msft_df)

Holding = 137410.0, VaR = -5361.47 over 1 day
Holding = 137410.0, modified VaR = -11557.92 over 1 day


In [48]:
historical_VaR(msft_df)

Holding = 137410.0, historical VaR = -4507.7902 tomorrow


In [49]:
monte_carlo_VaR(msft_df)

Holding = 137410.0 Monte Carlo VaR = -5244.0437 tomorrow
